# Credit Card Fraud Detection using a Deep Neural Network

For more information on the dataset look at the other file, if you havent already.

## Load Data

In [1]:
#Import relevant dependencies
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing as pp

#Loading data into a numpy array
raw_data = np.loadtxt('creditcard.csv', delimiter = ',')

#Excludes the targets
unscaled_inputs_all = raw_data[:,:-1]

#Last column stores the targets
targets_all = raw_data[:,-1]

## Balance the Dataset

In [2]:
#Targets are binary (1 or 0) so getting the sum of all target values is equivalent to how many 1s are in the dataset
num_one_targets = int(np.sum(targets_all))

#Our zero target counter
num_zero_targets = 0

#Once we have an equal amount of 0s and 1s we need to remove all the other data points and we will store the indices to be removed in a list
indices_to_remove = []

#Iterating through our dataset to load our indices_to_remove list
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0.:
        num_zero_targets += 1.
        if num_zero_targets > num_one_targets:
            indices_to_remove.append(i)  

#Storing the new balanced inputs and targets by removing all the unnecessary datapoints
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis = 0)

## Standardize Inputs

In [3]:
#Standardizes the inputs for more accuracy
scaled_inputs = pp.scale(unscaled_inputs_equal_priors)

## Shuffle the Data

In [4]:
#np.arange([start], stop) returns evenly spaced values in a given interval
shuffled_indices = np.arange(scaled_inputs.shape[0])
#np.random.shuffle(X) is a method that shuffles numbers in a given sequence
np.random.shuffle(shuffled_indices)

#Storing the new shuffled inputs and targets
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

## Split Data into Train, Test Subsets

In [6]:
#Count of all data points
samples_count = shuffled_inputs.shape[0]

#Storing the count of data points in each subset
train_samples_count = int(0.8*samples_count)
test_samples_count = int(0.2*samples_count)

train_inputs = shuffled_inputs[: train_samples_count]
train_targets = shuffled_targets[: train_samples_count]

test_inputs = shuffled_inputs[train_samples_count:]
test_targets = shuffled_targets[train_samples_count:]

#Shows how balanced the datasets are
print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)

398.0 787 0.5057179161372299
94.0 196 0.47959183673469385


In [7]:
np.savez('audiobooks_data_train', inputs = train_inputs, targets = train_targets)
np.savez('audiobooks_data_test', inputs = test_inputs, targets = test_targets)

## Loading the .npz Files

In [8]:
#Loading the data from the npz files
#We need to load the inputs and targets into their dedicated arrays
npz = np.load('audiobooks_data_train.npz')

train_inputs = npz['inputs']
train_targets = npz['targets']

npz = np.load('audiobooks_data_test.npz')

test_inputs = npz['inputs']
test_targets = npz['targets']

## Model

In [12]:
input_layer_size = 30
hidden_layer_size = 100
output_layer_size = 2

#Outlining the model similar to MNIST
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_layer_size, activation = 'softmax')
])

#Choose the optimizer and loss function
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

BATCH_SIZE = 10
NUM_OF_EPOCHS = 10

early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2)

model.fit(train_inputs,
         train_targets,
         batch_size = BATCH_SIZE,
         epochs = NUM_OF_EPOCHS, 
         callbacks = [early_stopping],
         validation_data = (test_inputs,
                           test_targets),
         verbose = 2)

Epoch 1/10
79/79 - 0s - loss: 0.2055 - accuracy: 0.9403 - val_loss: 0.0532 - val_accuracy: 0.9898
Epoch 2/10
79/79 - 0s - loss: 0.0412 - accuracy: 0.9898 - val_loss: 0.0197 - val_accuracy: 0.9949
Epoch 3/10
79/79 - 0s - loss: 0.0334 - accuracy: 0.9886 - val_loss: 0.0170 - val_accuracy: 0.9949
Epoch 4/10
79/79 - 0s - loss: 0.0082 - accuracy: 0.9975 - val_loss: 0.0144 - val_accuracy: 0.9898
Epoch 5/10
79/79 - 0s - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0091 - val_accuracy: 0.9949
Epoch 6/10
79/79 - 0s - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0106 - val_accuracy: 0.9898
Epoch 7/10
79/79 - 0s - loss: 9.8393e-04 - accuracy: 1.0000 - val_loss: 0.0080 - val_accuracy: 0.9949
Epoch 8/10
79/79 - 0s - loss: 6.1039e-04 - accuracy: 1.0000 - val_loss: 0.0083 - val_accuracy: 0.9949
Epoch 9/10
79/79 - 0s - loss: 4.6245e-04 - accuracy: 1.0000 - val_loss: 0.0075 - val_accuracy: 0.9949
Epoch 10/10
79/79 - 0s - loss: 3.3040e-04 - accuracy: 1.0000 - val_loss: 0.0075 - val_accuracy: 0.9949


In [13]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

7/7 [==============================] - 0s 2ms/step - loss: 0.0075 - accuracy: 0.9949


In [11]:
print('Test Loss: {0:.2f}. Test Accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test Loss: 0.02. Test Accuracy: 99.49%


## Additional Comments

Used the test data to validate the training data considering the small size of the dataset.